# Object Detection
> 객체 탐지 논문

- toc: true 
- badges: true
- comments: true
- categories: [python]
- image: images/

---


참고 사이트 : https://ganghee-lee.tistory.com/

<img src='img/Detection/공부순서.png' width="500" height="500">

# 1. R-CNN (2013)
***Rich feature hierarchies for accurate object detection and semantic segmentation***

원문 : https://arxiv.org/abs/1311.2524

<img src='img/Detection/rcnn/rcnn.png' width="500" height="500">

**" R-CNN 프로세스 "**

1. Image를 입력받는다.

2. Selective search 알고리즘에 의해 regional proposal output 약 2000개를 추출한다.

    (추출한 regional proposal output을 모두 동일 input size로 만들어주기 위해 warp 해준다.
    
    `?` 왜 동일 input size로 만들어 줄까? : 사실 Convolution Layer에는 input size가 고정이지 않다. 그러나 마지막 FC layer에서의 input size는 고정이므로 Convolution Layer에 입력에서부터 동일한 input size로 넣어주어 output size를 동일하게 하는 것)

    
3. 2000개의 warped image를 각각 CNN 모델에 넣는다.

4. 각각의 Convolution 결과에 대해 classification을 진행하여 결과를 얻는다.



위의 과정을 수행하기 위해 R-CNN은 세 가지 모듈로 나누어 놓았다.

1. Region Proposal : "Object가 있을법한 영역"을 찾는 모듈 (기존의 Sliding window방식의 비효율성 극복)
    
2. CNN : 각각의 영역으로부터 고정된 크기의 Feature Vector를 뽑아낸다.

3. SVM : Classification을 위한 선형 지도학습 모델

    `?` 왜 Classifier로 Softmax를 쓰지 않고 SVM을 사용했을까? : CNN fine-tuning을 위한 학습 데이터가 시기상 많지 않아서 Softmax를 적용시키면 오히려 성능이 낮아져 SVM을 사용

## 1. Region Proposal

<img src='img/Detection/rcnn/region.png' width="500" height="500">

R-CNN에서는 가장 먼저 Region Proposal 단계에서 "물체가 있을 법한 영역"을 찾는다.

이는 위에서 말했듯이 기존의 Sliding window방식의 비효율성을 극복하기 위한 것이다.

먼저 기존의 Sliding window가 무엇인지 살펴보자.

### Sliding Window

Sliding window방식은 이미지에서 물체를 찾기 위해 window의 (크기, 비율)을 임의로 마구 바꿔가면서
모든 영역에 대해서 탐색하는 것이다.

<img src='img/Detection/rcnn/sliding.png' width="500" height="500">

<center>좌 : 모든 영역에 대해 탐색 / 우 : 크기와 비율을 변형</center>

이렇게 임의의 (크기, 비율)로 모든 영역을 탐색하는 것은 너무 느리다.

따라서 R-CNN에서는 이 비효율성을 극복하기 위해 Selective search 알고리즘을 사용한다.

### Selective search

<img src='img/Detection/rcnn/selective.png' width="500" height="500">

1. 색상, 질감, 영역크기 등.. 을 이용해 non-object-based segmentation을 수행한다.

    이 작업을 통해 좌측 제일 하단 그림과 같이 많은 small segmented areas들을 얻을 수 있다.

2. Bottom-up 방식으로 small segmented areas들을 합쳐서 더 큰 segmented areas들을 만든다.

3. (2)작업을 반복하여 최종적으로 2000개의 region proposal을 생성한다.

Selective search알고리즘에 의해 2000개의 region proposal이 생성되면 이들을 모두 CNN에 넣기 전에
같은 사이즈로 warp시켜야한다. (CNN output 사이즈를 동일하게 만들기 위해 - For FC layer)

## 2. CNN

<img src='img/Detection/rcnn/cnn.png' width="500" height="500">

Warp작업을 통해 region proposal 모두 224x224 크기로 되면 CNN 모델에 넣는다.

여기서 CNN은 AlexNet의 거의 구조를 그대로 가져다 썼다.

최종적으로 CNN을 거쳐 각각의 region proposal로부터 4096-dimentional feature vector를 뽑아내고,

이를 통해 고정길이의 Feature Vector를 만들어낸다.

## 3. SVM

<img src='img/Detection/rcnn/svm.png' width="500" height="500">

CNN모델로부터 feature가 추출되면 Linear SVM을 통해 classification을 진행한다.

위에서 설명했듯이 Classifier로 softmax보다 SVM이 더 좋은 성능을 보였기 때문에 SVM을 채택했다.

SVM은 CNN으로부터 추출된 각각의 feature vector들의 점수를 class별로 매기고, 객체인지 아닌지,
객체라면 어떤 객체인지 등을 판별하는 역할을 하는 Classifier이다.

`+` Selective search로 만든 bounding box는 정확하지 않기 때문에 물체를 정확히 감싸도록 조정해주는 bounding box regression(선형회귀 모델)이 존재한다.

## 단점

1. 여기서 selective search로 2000개의 region proposal을 뽑고 각 영역마다 CNN을 수행하기 때문에

    CNN연산 * 2000 만큼의 시간이 걸려 수행시간이 매우 느리다. 

 

2. CNN, SVM, Bounding Box Regression 총 세가지의 모델이 multi-stage pipelines으로 한 번에 학습되지 않는다.

    각 region proposal 에 대해 ConvNet forward pass를 실행할때 연산을 공유하지 않기에
    end-to-end 로 학습할 수 없다.

    따라서 SVM, bounding box regression에서 학습한 결과가 CNN을 업데이트 시키지 못한다.
    
    (* bounding box regression은 CNN을 거치기 전의 region proposal 데이터가 input으로 들어가고 SVM은 CNN을 거친 후의 feature map이 input으로 들어가기에 연산이 겹치지 않는다.)
    
"그리고 이 두가지 문제를 RoI pooling으로 해결한 Fast R-CNN이 나오게 된다."

---

# 2. Fast R-CNN (2014)
***Fast R-CNN***

원문 :  https://arxiv.org/abs/1504.08083

## Introduction

Fast R-CNN은 이전 R-CNN의 한계점을 극복하고자 나왔다. R-CNN는 이전 글에서 언급했듯이

1) RoI (Region of Interest) 마다 CNN연산을 함으로써 속도저하

2) multi-stage pipelines으로써 모델을 한번에 학습시키지 못함

다음과 같은 한계점들이 있었다.

 

그리고 Fast R-CNN에서는 다음 두 가지를 통해 위 한계점들을 극복했다.

1) RoI pooling

2) CNN 특징 추출부터 classification, bounding box regression까지 하나의 모델에서 학습

## Fast R-CNN

<img src='img/Detection/fastrcnn/fastrcnn.png' width="500" height="500">

**"Fast R-CNN 프로세스"**

1-1. R-CNN에서와 마찬가지로 Selective Search를 통해 RoI를 찾는다.

1-2. 전체 이미지를 CNN에 통과시켜 feature map을 추출한다.

2. Selective Search로 찾았었던 RoI를 feature map크기에 맞춰서 projection시킨다.

3. projection시킨 RoI에 대해 RoI Pooling을 진행하여 고정된 크기의 feature vector를 얻는다.

4. feature vector는 FC layer를 통과한 뒤, 두 브랜치로 나뉘게 된다.

5-1. 하나는 softmax를 통과하여 RoI에 대해 object classification을 한다.

5-2. bounding box regression을 통해 selective search로 찾은 box의 위치를 조정한다. 

Fast R-CNN의 가장 핵심적인 아이디어는 RoI Pooling이다.

R-CNN에서 CNN output이 FC layer의 input으로 들어가야했기 때문에 CNN input을 동일 size로 맞춰줘야 했다.

따라서 원래 이미지에서 추출한 RoI를 crop, warp을 통해 동일 size로 조정했었다.

그러나 실제로 "FC layer의 input이 고정인거지 CNN input은 고정이 아니다"

따라서 CNN에는 입력 이미지 크기, 비율 관계없이 input으로 들어갈 수 있고

FC layer의 input으로 들어갈때만 size를 맞춰주기만 하면된다.

여기서 Spatial Pyramid Pooling(SPP)이 제안된다.

## Spatial Pyramid Pooling(SPP)

<img src='img/Detection/fastrcnn/spp.png' width="500" height="500">

SPP에서는 먼저 이미지를 CNN에 통과시켜 feature map을 추출한다.

그리고 미리 정해진 4x4, 2x2, 1x1 영역의 피라미드로 feature map을 나눠준다. 피라미드 한칸을 bin이라 한다.

bin내에서 max pooling을 적용하여 각 bin마다 하나의 값을 추출하고,

최종적으로 피라미드 크기만큼 max값을 추출하여 3개의 피라미드의 결과를 쭉 이어붙여 고정된 크기 vector를 만든다.

 

정리하자면,

4x4, 2x2, 1x1 세 가지 피라미드가 존재하고, max pooling을 적용하여 각 피라미드 크기에 맞게 max값을 뽑아낸다.

각 피라미드 별로 뽑아낸 max값들을 쭉 이어붙여 고정된 크기 vector를 만들고 이게 FC layer의 input으로 들어간다.

 

따라서 CNN을 통과한 feature map에서 2천개의 region proposal을 만들고 region proposal마다

SPPNet에 집어넣어 고정된 크기의 feature vector를 얻어낸다. 

**이 작업을 통해 모든 2천개의 region proposal마다 해야했던 2천번의 CNN연산이 1번으로 줄었다.**

## RoI Pooling

다시 돌아와 Fast R-CNN에서 이 SPP가 적용되는 것을 보면 다음과 같다.

<img src='img/Detection/fastrcnn/fastrcnn2.png' width="500" height="500">

실제로 Fast R-CNN에서는 1개의 피라미드를 적용시킨 SPP로 구성되어있다. 또한 피라미드의 사이즈는 7x7이다.

Fast R-CNN에서 적용된 1개의 피라미드 SPP로 고정된 크기의 feature vector를 만드는과정을 **RoI Pooling**이라 한다.

<img src='img/Detection/fastrcnn/roi.png' width="500" height="500">

<center>RoI Pooling</center>

Fast R-CNN에서 먼저 입력 이미지를 CNN에 통과시켜 feature map을 추출한다.

그 후 이전에 미리 Selective search로 만들어놨던 RoI(=region proposal)을 feature map에 projection시킨다.

위 그림의 가장 좌측 그림이 feature map이고 그 안에 hxw크기의 검은색 box가 투영된 RoI이다.

 

(1) 미리 설정한 HxW크기로 만들어주기 위해서 (h/H) * (w/H) 크기만큼 grid를 RoI위에 만든다.

(2) RoI를 grid크기로 split시킨 뒤 max pooling을 적용시켜 결국 각 grid 칸마다 하나의 값을 추출한다.

위 작업을 통해 feature map에 투영했던 hxw크기의 RoI는 HxW크기의 고정된 feature vector로 변환된다.

 

이렇게 RoI pooling을 이용함으로써

"원래 이미지를 CNN에 통과시킨 후 나온 feature map에 이전에 생성한 RoI를 projection시키고

이 RoI를 FC layer input 크기에 맞게 고정된 크기로 변형할 수가 있다"

따라서 더이상 2000번의 CNN연산이 필요하지 않고 1번의 CNN연산으로 속도를 대폭 높일 수 있었다.

## end-to-end : Trainable

다음은 R-CNN의 두번째 문제였던 multi-stage pipeline으로 인해 3가지 모델을 따로 학습해야했던 문제이다.

R-CNN에서는 CNN을 통과한 후 각각 서로다른 모델인 SVM(classification), bounding box regression(localization)안으로 들어가 forward됐기 때문에 연산이 공유되지 않았다.

(* bounding box regression은 CNN을 거치기 전의 region proposal 데이터가 input으로 들어가고 SVM은 CNN을 거친 후의 feature map이 input으로 들어가기에 연산이 겹치지 않는다.)

<img src='img/Detection/fastrcnn/fastrcnn.png' width="500" height="500">

<center>Fast R-CNN</center>

그러나 위 그림을 다시보면 RoI Pooling을 추가함으로써 이제 RoI영역을

CNN을 거친후의 feature map에 투영시킬 수 있었다. 

따라서 동일 data가 각자 softmax(classification), bbox regressor(localization)으로 들어가기에 연산을 공유한다.

이는 이제 모델이 end-to-end로 한 번에 학습시킬 수 있다는 뜻이다.

## 결론

RoI Pooling을 하나 추가함으로써 

(1) CNN후에 region proposal 연산 - 2000xCNN연산 → 1번의 CNN연산

(2) 변경된 feature vector가 결국 기존의 region proposal을 projection시킨 후 연산한 것이므로

해당 output으로 classification과 bbox regression도 학습 가능의 성과를 이룰 수 있었다.

**그러나 여전히 Fast R-CNN에서도 R-CNN에서와 마찬가지로 RoI를 생성하는 Selective search알고리즘은 CNN외부에서 진행되므로 이 부분이 속도의 bottleneck이다.** 

따라서 이 RoI 생성마저 CNN내부에서 함으로써 더욱 빠르면서 정확한 region proposal을 생성한 Faster R-CNN이 나오게 된다.

---
# 3. Faster R-CNN (2015)
***Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks***

원문 :  https://arxiv.org/abs/1506.01497

## introduction

**R-CNN**에서는 3가지 모듈 (region proposal, classification, bounding box regression)을 각각 따로따로 수행한다.

(1)region proposal 추출 → 각 region proposal별로 CNN 연산 → (2)classification, (3)bounding box regression

 



**Fast R-CNN**에서는 region proposal을 CNN level로 통과시켜 classification, bounding box regression을 하나로 묶었다.

(1)region proposal 추출 → 전체 image CNN 연산 → RoI projection, RoI Pooling → (2)classification, bounding box regression

그러나 여전히 region proposal인 Selective search 알고리즘을 CNN외부에서 연산하므로 RoI 생성단계가 병목이다.

따라서 Faster R-CNN에서는 detection에서 쓰인 conv feature을 RPN에서도 공유해서

RoI생성역시 CNN level에서 수행하여 속도를 향상시킨다. 

"Region Proposal도 Selective search 쓰지말고 CNN - (classification | bounding box regression) 이 네트워크 안에서 같이 해보자!"

## Faster R-CNN

Selective search가 느린이유는 cpu에서 돌기 때문이다.

따라서 Region proposal 생성하는 네트워크도 gpu에 넣기 위해서 Conv layer에서 생성하도록 하자는게 아이디어이다.

 

Faster R-CNN은 한마디로 RPN + Fast R-CNN이라할 수 있다.

Faster R-CNN은 Fast R-CNN구조에서 conv feature map과 RoI Pooling사이에 RoI를 생성하는

Region Proposal Network가 추가된 구조이다.

<img src='img/Detection/fasterrcnn/faster.png' width="500" height="500">

<center>RPN + Fast R-CNN</center>

그리고 Faster R-CNN에서는 RPN 네트워크에서 사용할 CNN과

Fast R-CNN에서 classification, bbox regression을 위해 사용한 CNN 네트워크를 공유하자는 개념에서 나왔다.

<img src='img/Detection/fasterrcnn/share.png' width="500" height="500">

<center>CNN Sharing (RPN & Detector)</center>

결국 위 그림에서와 같이 CNN을 통과하여 생성된 conv feature map이 RPN에 의해 RoI를 생성한다.

이렇게 feature map에 RoI가 투영되고 나면 FC layer에 의해 classification과 bbox regression이 수행된다.

<img src='img/Detection/fasterrcnn/process.png' width="500" height="500">

위 그림에서 보다시피 마지막에 FC layer를 사용하기에 input size를 맞춰주기 위해 RoI pooling을 사용한다.

## RPN (Region Proposal Network)

<img src='img/Detection/fasterrcnn/rpn.png' width="500" height="500">

RPN의 input 값은 이전 CNN 모델에서 뽑아낸 feature map이다.

Region proposal을 생성하기 위해 feature map위에 nxn window를 sliding window시킨다. 

이때, object의 크기와 비율이 어떻게 될지모르므로 k개의 anchor box를 미리 정의해놓는다.

이 anchor box가 bounding box가 될 수 있는 것이고 미리 가능할만한 box모양 k개를 정의해놓는 것이다.

여기서는 가로세로길이 3종류 x 비율 3종류 = 9개의 anchor box를 이용한다.

이 단계에서 9개의 anchor box를 이용하여 classification과 bbox regression을 먼저 구한다. (For 학습)

먼저, CNN에서 뽑아낸 feature map에 대해 3x3 conv filter 256개를 연산하여 depth를 256으로 만든다.

그 후 1x1 conv 두개를 이용하여 각각 classification과 bbox regression을 계산한다.

1x1 convolution은 fully connected layer와 같다고 한다.

RPN에서 이렇게 1x1 convolution을 이용하여 classification과 bbox regression을 계산하는데

이때 네트워크를 가볍게 만들기 위해 binary classification으로 bbox에 물체가 있나 없나만 판단한다.

무슨 물체인지 classification하는 것은 마지막 classification 단계에서 한다.

RPN단계에서 classification과 bbox regression을 하는 이유는 결국 학습을 위함이다.

IoU가 0.7보다 크거나, 한 지점에서 모든 anchor box중 가장 IoU가 큰 anchor box는 positive example로 만든다.

IoU가 0.3보다 작으면 object가 아닌 background를 뜻하므로 negative example로 만들고

이 사이에 있는 IoU에 대해서는 애매한 값이므로 학습 데이터로 이용하지 않는다.

## Non-Maximu Suppression  NMS알고리즘

Faster R-CNN에 대한 학습이 완료된 후 RPN모델을 예측시키며 하마 한 객체당 여러 proposal값이 나올 것이다.

이 문제를 해결하기 위해 NMS알고리즘을 사용하여 proposal의 개수를 줄인다.

NMS알고리즘은 다음과 같다.

1. box들의 score(confidence)를 기준으로 정렬한다.

2. score가 가장 높은 box부터 시작해서 다른 모든 box들과 IoU를 계산해서 0.7이상이면 같은 객체를 detect한 box라고
생각할 수 있기 때문에 해당 box는 지운다. 

3. 최종적으로 각 object별로 score가 가장 높은 box 하나씩만 남게 된다.

<img src='img/Detection/fasterrcnn/before.png' width="500" height="500">

<center>NMS 전</center>

<img src='img/Detection/fasterrcnn/after.png' width="500" height="500">

<center>NMS 후</center>